⬇️ **Вставьте в нижнюю ячейку ссылку на папку с товарными декларациями и запустите код** ⬇️

In [2]:
pdf_folder = ''

<br><br>
### Шаг 1. Загрузка библиотек и профильной информации

In [4]:
# уберите при необходимости комментирование и установите нужные библиотеки

#!pip install pdfminer.six -q
#!pip install pdfplumber -q
#!pip install PyPDF2 -q
#!pip install pillow -q
#!pip install pytesseract -q
#!pip install pdf2image -q
#!pip install pandas -q

In [5]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTFigure
import pdfplumber
import PyPDF2
from PIL import Image
import pytesseract
import os
import re
import pandas as pd
from copy import deepcopy
from pdf2image import convert_from_path
from IPython.display import display

In [6]:
# коды товаров согласно постановлению

product_codes_4 = [
    "3919", "3920", "3922", "3923", "3924", "4009", "4010", "4011", "4012", "4013",
    "4202", "4405", "4415", "4421", "4802", "4805", "4806", "4808", "4810", "4811",
    "4817", "4819", "4820", "4821", "4822", "4823", "4902", "4911", "5603", "5607",
    "5608", "5701", "5702", "5703", "5704", "5804", "5808", "5810", "6110", "6111",
    "6112", "6114", "6115", "6116", "6101", "6102", "6103", "6104", "6105", "6106",
    "6107", "6108", "6109", "6117", "6201", "6202", "6203", "6204", "6205", "6206",
    "6207", "6208", "6209", "6210", "6211", "6212", "6213", "6214", "6215", "6217",
    "6301", "6302", "6303", "6304", "6306", "6310", "6401", "6402", "6403", "6404",
    "6405", "6406", "6506", "7003", "7004", "7007", "7009", "7321", "7322", "8404",
    "8414", "8415", "8418", "8419", "8423", "8425", "8443", "8450", "8451", "8452",
    "8467", "8470", "8471", "8472", "8476", "8481", "8504", "8506", "8508", "8509",
    "8516", "8517", "8519", "8521", "8523", "8524", "8525", "8527", "8528", "8531",
    "8534", "8537", "8539", "8542", "9005", "9006", "9007", "9008", "9010", "9013",
    "9014", "9015", "9022", "9024", "9025", "9026", "9027", "9028", "9029", "9030",
    "9031", "9032", "9101", "9102", "9103", "9105", "9404", "9405", "9613"
]

product_codes_5 = ["41699"]

product_codes_6 = [
    "271019", "392330", "392590", "401699", "480700", "481032", "590900", "621132", "621133",
    "621142", "621143", "630710", "630790", "650500", "701328", "701349", "701399", "731010", 
    "731021", "731029", "842310", "842430", "847160", "847170", "850710", "850720", "850730",  
    "851310", "851610", "851629", "851660", "851679", "851981", "852341", "852660", "852910", 
    "853922", "853929", "702000", "441520"
]

product_codes_7 = ["8525000"]

product_codes_8 = ["94051100", "94051900", "94054100", "94054200", "94054900"]

product_codes_9 = [
    "401512000", "401610000", "401693000", "480840000", "481930000", "842121000", "842519000", "847050000",
    "850750000", "850760000", "850780000", "851631000", "851711000", "910400000", "940541001", "940541002",
    "940542001", "940549001", "621149000", "630231000"
]

product_codes_10 = [
    "6302100001", "6302210000", "5805000000", "6307200000", "5704570500", "5609000000", "5910000000", "5804630790", "6203221000", "6203231000", 
    "6203291100", "6203293000", "6203299000", "6203110000", "6203120000", "6203191000", "6203193000", "6203199000", "6203310000", "6203321000", 
    "6203331000", "6203391100", "6203399000", "6203411000", "6203421100", "6203431100", "6203491100", "6203425100", "6203433100", "6203493100", 
    "6204210000", "6204221000", "6204231000", "6204291100", "6204299000", "6204110000", "6204120000", "6204130000", "6204191000", "6204199000", 
    "6204310000", "6204321000", "6204331000", "6204391100", "6204399000", "6204618500", "6204625100", "6204633100", "6204693100", "6211390000", 
    "6104199009", "6309000000", "6307909800", "6504000000", "6505003000", "4911990000", "4909000000", "4910000000", "4823610000", "4823691000", 
    "4823699000", "4012909000", "4010310000", "4010320000", "4010330000", "4010340000", "4010350000", "4010360000", "4010390000", "4015190000", 
    "4016910000", "4016950000", "4016930000", "4016920000", "3926200000", "3926100000", "3926400000", "9505109000", "7013990000", "8471500000", 
    "8525500000", "9014100000", "9101210000", "9101290000", "9101990000", "9101110000", "9101190000", "9101910000", "9002110000", "9006400000", 
    "9007200000", "9008500000", "9010500000", "9010600000", "9013200000", "8523210000", "8422110000", "6301100000", "8510100000", "8510300000", 
    "8510200000", "8516320000", "8516330000", "8516400000", "8516500000", "8543400000", "8425110000", "8470100000", "8415820000", "8507800001", 
    "9405310000", "9405390000", "8421230000", "8421310000", "3025200000", "3925300000", "4416000000", "4805300000", "4819400000", "4819100000", 
    "4819500000", "4811510009"
]

In [7]:
packaging_codes = [
"BA", "BB", "BC", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BK", "BL", "BM", "BN", "BO", "BP", "BQ", "BR", "BS", "BT", "BU", "BV", "BX", "BY", "BZ",
"CA", "CB", "CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CP", "CR", "CS", "CT", "CU", "CV", "CW", "CX", "CY", "CZ",
"DA", "DB", "DC", "DG", "DH", "DI", "DJ", "DK", "DL", "DM", "DN", "DP", "DR", "DS", "DT", "DU", "DV", "DW", "DX", "DY",
"ED", "EE", "EF", "EG", "EH", "EI",
"FC", "FD", "FE", "FI", "FL", "FO", "FP", "FR", "FT", "FX",
"GB", "GL", "GR",
"HA", "HB", "HC", "HG", "HN", "HR",
"IA", "IB", "IC", "ID", "IE", "IF", "IG", "IH", "IK", "IN", "IZ",
"JB", "JC", "JG", "JR", "JT", "JY",
"KG", "KI",
"LE", "LG", "LT", "LU", "LV", "LZ",
"MA", "MB", "MC", "ME", "MR", "MS", "MT", "MW", "MX",
"NA", "NE", "NF", "NG", "NS", "NT", "NU", "NV",
"PA", "PB", "PC", "PD", "PE", "PF", "PG", "PH", "PI", "PJ", "PK", "PL", "PN", "PO", "PP", "PR", "PT", "PU", "PV", "PX", "PY", "PZ",
"QA", "QB", "QC", "QD", "QF", "QG", "QH", "QJ", "QK", "QL", "QM", "QN", "QP", "QQ", "QR", "QS",
"RD", "RG", "RJ", "RK", "RL", "RO", "RT", "RZ",
"SA", "SB", "SC", "SD", "SE", "SH", "SI", "SK", "SL", "SM", "SO", "SP", "SS", "ST", "SU", "SV", "SW", "SX", "SY", "SZ",
"TB", "TC", "TD", "TE", "TG", "TI", "TK", "TL", "TN", "TO", "TR", "TS", "TT", "TU", "TV", "TW", "TY", "TZ",
"UC",
"VA", "VG", "VI", "VK", "VL", "VO", "VP", "VQ", "VR", "VS", "VY",
"WA", "WB", "WC", "WD", "WF", "WG", "WH", "WJ", "WK", "WL", "WM", "WN", "WP", "WQ", "WR", "WS", "WT", "WU", "WV", "WW", "WX", "WY", "WZ",
"XA", "XB", "XC", "XD", "XF", "XG", "XH", "XJ", "XK",
"YA", "YB", "YC", "YD", "YF", "YH", "YJ", "YK", "YL", "YM", "YN", "YP", "YQ", "YR", "YS", "YT", "YV", "YW", "YX", "YY", "YZ",
"ZA", "ZB", "ZC", "ZD", "ZF", "ZG", "ZH", "ZJ", "ZK", "ZL", "ZM", "ZN", "ZP", "ZQ", "ZR", "ZS", "ZT", "ZU", "ZV", "ZW", "ZX", "ZY", "ZZ"
]

In [8]:
# коды таможенных процедур

ktp = ["1000000", "2300000", "4000000", "0700000", "5300000", "3100000", "4200000", "6100000", "7100000", "7600000"]

<br><br>
### Шаг 2. Извлечение текста из pdf-файлов

In [ ]:
def извлечь_текст_из_элемента(element):
    текст_строки = element.get_text()
    форматы_строки = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            for char in text_line:
                if isinstance(char, LTChar):
                    форматы_строки.append((char.fontname, char.size))
    уникальные_форматы = list(set(форматы_строки))
    return текст_строки, уникальные_форматы

def обрезать_изображение(element, page_obj):
    left, bottom, right, top = element.x0, element.y0, element.x1, element.y1
    cropped_page = deepcopy(page_obj)
    cropped_page.mediabox.lower_left = (left, bottom)
    cropped_page.mediabox.upper_right = (right, top)
    writer = PyPDF2.PdfWriter()
    writer.add_page(cropped_page)
    temp_pdf = 'temp_cropped.pdf'
    with open(temp_pdf, 'wb') as f:
        writer.write(f)
    return temp_pdf

def pdf_в_изображения(pdf_path):
    return convert_from_path(pdf_path)

def изображение_в_текст(image_path):
    img = Image.open(image_path)
    return pytesseract.image_to_string(img)

def удалить_временные_файлы(*filenames):
    for file in filenames:
        if os.path.exists(file):
            os.remove(file)

# Сохраняем текст всех страниц всех файлов
text_per_page_all_files = {}
весь_текст = ''

for filename in os.listdir(pdf_folder):
    if filename.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)

        with pdfplumber.open(pdf_path) as pdf:
            pdf_reader = PyPDF2.PdfReader(pdf_path)

            for page_num, page_layout in enumerate(extract_pages(pdf_path)):
                page_obj = pdf_reader.pages[page_num]
                page_content = []

                elements_sorted = sorted([(el.y1, el) for el in page_layout._objs], key=lambda x: x[0], reverse=True)

                for _, element in elements_sorted:
                    if isinstance(element, LTTextContainer):
                        text_line, _ = извлечь_текст_из_элемента(element)
                        clean_text = text_line.replace('\n', '').replace(' ', '').replace(' ', '')
                        page_content.append(clean_text)
                    elif isinstance(element, LTFigure):
                        cropped_pdf_path = обрезать_изображение(element, page_obj)
                        images = pdf_в_изображения(cropped_pdf_path)
                        if images:
                            image_path = 'temp_image.png'
                            images[0].save(image_path, 'PNG')
                            ocr_text = изображение_в_текст(image_path)
                            clean_ocr = ocr_text.replace('\n', '').replace(' ', '').replace(' ', '')
                            page_content.append(clean_ocr)
                            удалить_временные_файлы(cropped_pdf_path, image_path)

                key = f'{filename}__Page_{page_num + 1}'
                text_per_page_all_files[key] = page_content
                весь_текст += ''.join(page_content)

# 🔍 Выводим постраничный текст в правильном порядке:
def sort_key(key):
    # Разделяем 'filename.pdf__Page_3' на части
    match = re.match(r'(.+?)__Page_(\d+)', key)
    if match:
        filename_part = match.group(1).lower()
        page_num = int(match.group(2))
        return (filename_part, page_num)
    return (key, 0)  # fallback

for key in sorted(text_per_page_all_files.keys(), key=sort_key):
    print(f"\n📄 {key}")
    print(''.join(text_per_page_all_files[key]))

<br><br>
### Шаг 3. Разбивка текста на блоки (информация из блока №0 попадет только в строку с индексом 0 и т.д.)

In [ ]:
блоки_текста = []
pattern_start = 'грузовыеместа'
pattern_end = 'кодмос'
pattern_кодтовара = re.compile(r'кодтовара\d{10}', re.IGNORECASE)

for key in sorted(text_per_page_all_files.keys(), key=sort_key):
    текст_страниц = ''.join(text_per_page_all_files[key]).lower()
    имя_файла = key.split('__Page_')[0]  # извлекаем имя файла без страницы

    start_positions = [m.start() for m in re.finditer(pattern_start, текст_страниц)]
    end_positions = [m.start() for m in re.finditer(pattern_end, текст_страниц)]

    for start_index in start_positions:
        end_index = next((end for end in end_positions if end > start_index), None)
        if end_index is None:
            continue

        raw_block = текст_страниц[start_index:end_index]

        # Проверяем наличие кода товара и "кол-во"
        if pattern_кодтовара.search(raw_block) and "кол-во" in raw_block:
            блоки_текста.append((raw_block, имя_файла))
        else:
            lookback = 1
            while start_index - lookback >= 0:
                extended_block = текст_страниц[start_index - lookback:end_index]
                if pattern_кодтовара.search(extended_block) and "кол-во" in extended_block:
                    блоки_текста.append((extended_block, имя_файла))
                    break
                lookback += 1

# 🔍 Вывод извлечённых блоков
print(f"\n📦 Найдено блоков: {len(блоки_текста)}\n")
for i, (блок, файл) in enumerate(блоки_текста, start=1):
    print(f"🔹 Блок #{i} из файла {файл}:\n{блок}\n{'-'*80}")

<br><br>
### Шаг 4. Извлечение информации из блоков текста и создание excel-таблицы.

⬇️ **Код товара** ⬇️

In [ ]:
def извлечь_код_товара(блок_clean):
    match = re.search(r'кодтовара(\d{10})', блок_clean)
    if not match:
        return None, None  # Возвращаем и код, и наличие в РОП

    код = match.group(1)
    if код[:4] in product_codes_4 or \
       код[:5] in product_codes_5 or \
       код[:6] in product_codes_6 or \
       код[:7] in product_codes_7 or \
       код[:8] in product_codes_8 or \
       код[:9] in product_codes_9 or \
       код[:10] in product_codes_10:
        return код, "да"
    else:
        return код, "нет"

⬇️ **Коды упаковок и их количество** ⬇️

In [ ]:
def извлечь_все_коды_упаковок(блок_clean):
    """Возвращает словарь кодов упаковки и их количеств. Если количество не указано — 0."""
    паттерн_количество = r'([A-Za-z]{2})-(\d+),'
    паттерн_только_код = r'\b([A-Za-z]{2})\b'

    def условия(текст):
        match = re.search(r'веснетто\(кг\)', текст, flags=re.IGNORECASE)
        if match:
            yield текст[match.end():match.end() + 180]

        match = re.search(r'cn:', текст, flags=re.IGNORECASE)
        if match:
            yield текст[max(0, match.start() - 30):match.start()]

        match = re.search(r'поддон', текст, flags=re.IGNORECASE)
        if match:
            yield текст[max(0, match.start() - 30):match.end() + 30]

        match = re.search(r'\(1-частьместа\)', текст, flags=re.IGNORECASE)
        if match:
            yield текст[match.end():match.end() + 20]

    коды_с_количеством = {}
    коды_встретились = set()

    for подстрока in условия(блок_clean):
        # Сначала ищем пары код-число
        for код, число in re.findall(паттерн_количество, подстрока, flags=re.IGNORECASE):
            код = код.upper()
            if код in packaging_codes:
                коды_с_количеством[код] = int(число)
                коды_встретились.add(код)

        # Затем ищем коды без количества
        for код in re.findall(паттерн_только_код, подстрока, flags=re.IGNORECASE):
            код = код.upper()
            if код in packaging_codes and код not in коды_с_количеством:
                коды_с_количеством[код] = 0

    return коды_с_количеством if коды_с_количеством else None

⬇️ **Вес брутто** ⬇️

In [ ]:
def извлечь_вес_брутто(блок_clean):
    match = re.search(r'([\d]+\.\d+)(?=38веснетто)', блок_clean)
    return float(match.group(1)) if match else None

⬇️ **Вес нетто** ⬇️

In [ ]:
def извлечь_вес_нетто(блок_clean, ktp):
    ktp_pattern = '|'.join(ktp)

    # 1. Пытаемся найти по основному шаблону с учетом ktp после веса
    match = re.search(
        r'веснетто\(кг\)([\d]+\.\d+|[\d]+)(?:/([\d]+\.\d+))?(?=' + ktp_pattern + ')',
        блок_clean
    )
    
    # 2. Если не нашли, ищем по шаблону без проверки ktp
    if not match:
        match = re.search(
            r'веснетто\(кг\)([\d]+\.\d+|[\d]+)(?:/([\d]+\.\d+))?',
            блок_clean
        )
    
    # 3. Если нашли по шаблонам выше — возвращаем
    if match:
        if match.group(2):
            return round(float(match.group(2)), 2)
        return round(float(match.group(1)), 2)

    # 4. Дополнительное условие: число между ")" и "40общаядекларация"
    match = re.search(r'\)(\d{2,6})40общаядекларация', блок_clean)
    if match:
        try:
            return round(float(match.group(1)), 2)
        except ValueError:
            pass

    return None

⬇️ **Вес упаковки** ⬇️

In [ ]:
def рассчитать_вес_упаковки(вес_брутто, вес_нетто):
    if вес_брутто is not None and вес_нетто is not None:
        return round(вес_брутто - вес_нетто, 3)
    return None

⬇️ **Количество товаров** ⬇️

In [ ]:
def извлечь_количество(блок_clean):
    match = re.search(r'дополнит\.единицы\s*([\d]+(?:\.\d+)?)\s*/', блок_clean)
    if match:
        return float(match.group(1))
    elif re.search(r'дополнит\.единицы\s*42ценатовара', блок_clean):
        паттерн = r'кол-во\s*([\d]+(?:\.\d+)?)(?=[A-Za-zА-Яа-я])'
        найденные_числа = re.findall(паттерн, блок_clean)
        числа_как_float = [float(x) for x in найденные_числа]
        if числа_как_float:
            return round(sum(числа_как_float), 3 if len(числа_как_float) > 1 else 2)
    return None

⬇️ **Финальный цикл** ⬇️

In [ ]:
rows = []

for блок_текста, имя_файла in блоки_текста:
    блок_clean = блок_текста.lower()

    код_товара, наличие_в_роп = извлечь_код_товара(блок_clean)
    if not код_товара:
        continue

    вес_брутто = извлечь_вес_брутто(блок_clean)
    вес_нетто = извлечь_вес_нетто(блок_clean, ktp)
    вес_упаковки = рассчитать_вес_упаковки(вес_брутто, вес_нетто)
    количество = извлечь_количество(блок_clean)
    коды_и_количества = извлечь_все_коды_упаковок(блок_clean)

    упаковка_рк = упаковка_ст = упаковка_рх = упаковка_иная = 0

    if коды_и_количества:
        for код, число in коды_и_количества.items():
            if код == 'PK':
                упаковка_рк += число
            elif код == 'CT':
                упаковка_ст += число
            elif код == 'PX':
                упаковка_рх += число
            else:
                упаковка_иная += число

    rows.append({
        'Наименование файла': имя_файла,
        'Код товара': код_товара,
        'Наличие в РОП': наличие_в_роп,
        'Вес брутто': вес_брутто,
        'Вес нетто': вес_нетто,
        'Кол-во товаров': количество,
        'Вес упаковок (кг)': вес_упаковки,
        'Упаковка РК': упаковка_рк,
        'Упаковка СТ': упаковка_ст,
        'Упаковка РХ': упаковка_рх,
        'Упаковка иная': упаковка_иная
    })

# ---- Анализ и сохранение ----

df = pd.DataFrame(rows)

условие_пропуски = df.isnull().any(axis=1)
условие_пустая_тара = (df[['Упаковка РК', 'Упаковка СТ', 'Упаковка РХ', 'Упаковка иная']].sum(axis=1) == 0)

df_для_частного_рассмотрения = df[условие_пропуски | условие_пустая_тара]
df_основные = df[~(условие_пропуски | условие_пустая_тара)]

print(f"🟩 Корректные значения (всего: {len(df_основные)} строк):")
display(df_основные.head())

print(f"\n🟥 Значения для частного рассмотрения (всего: {len(df_для_частного_рассмотрения)} строк):")
display(df_для_частного_рассмотрения.head())

df_основные.to_excel('результаты_деклараций.xlsx', index=False)
df_для_частного_рассмотрения.to_excel('результаты_для_частного_рассмотрения.xlsx', index=False)